# Exploiting Data Skew

In the previous lecture, we showed how to exploit redundancy by building a lookup table that mapped one of 4 possible strings to a 2-bit key:

| Color | key |
|-------|-------|
| Red   | 00 |
| Green  | 01|
| Blue  | 10|
| Black  | 11|

Suppose, we new that almost all of the colors in our list were Red. This scheme is a bit wasteful because we allocate the same number of bits to the most popular colors as the least. Is there a way to dynamically adjust our encoding to use less space to store strings that we know will show up more often.

This is not a niche problem. Almost all real-world data is skewed for example the characters in the engligh language follow this distribution: 
![title](https://upload.wikimedia.org/wikipedia/commons/thumb/d/d5/English_letter_frequency_%28alphabetic%29.svg/1024px-English_letter_frequency_%28alphabetic%29.svg.png)

## A Naive Approach

Let's try manually adjust our encoding to scale with popularity. Consider the lookup table before (with the strings annotated with popularity). We sort the strings from most to least popular, assign them a binary key, and then prune the leading zeros.

| Color | key |
|-------|-------|
| Red (0.8)   | 0 |
| Green (0.02)  |11|
| Blue (0.15)  | 10|
| Black (0.3)  | 1|

Our previous dictionary encoding approach requires 2-bits per string on average. Let's calculate the average (or expected) storage cost for this new encoding:

$$0.8*1 + 0.3*1 + 0.15 * 2 + 0.02 * 2 = 1.44$$

This means that simply changing the encoding compresses the data by about 40%. But, what goes wrong with this approach? Suppose, we retrieved the bits 10--how do we decode that? Is it "Black, Red" or is it "Blue". This is the core issue with this naive approach that there is amiguity in the decoding.

## Prefix-Free Codes
A careful reader might note that we could side-step this problem (and still get a compression benefit) by encoding the data slightly differently:

| Color | key |
|-------|-------|
| Red (0.8)   | 0 |
| Green (0.02)  |101|
| Blue (0.15)  | 11|
| Black (0.3)  | 10|

$$0.8*1 + 0.3*2 + 0.15 * 2 + 0.02 * 3 = 1.76$$

If read the data left to right, there is no longer any ambiguity 0101111101 => "0,Red", "10,Black", "11,Blue", "11,Blue", "101 Green". We simply scan the list until we find a "full" key in our lookup table. This works because each of the keys is prefix-free. A prefix-free encoding is a key that requires that there is no whole code word in the system that is a prefix (initial segment) of any other code word in the system. Luckily, we don't have to design such schemes by hand. There are algorithms that can find very efficient prefix-free encodings. 

## Huffman Coding
Huffman's encodes data by building a binary tree where the leaves are strings. The basic strategy is to go bottom up. Keep on grouping the lowest probability strings into common prefixes until there are none left:

![title](https://upload.wikimedia.org/wikipedia/commons/thumb/7/74/Huffman_coding_example.svg/1920px-Huffman_coding_example.svg.png)

We can easily express this algorithm in code. The first step is to write the bottom up algorithm that builds the tree of symbols.

In [11]:
#expect input of the form [(.95,['s1']), (.05,['s2'])]

def build_tree(strings):
    '''Given a set of strings and their associated
       probabilities build a tree with nested lists
    '''
    
    while len(strings) > 1:
        
        strings.sort() #sort by 
        
        p1,s1 = strings[0] #get lowest
        p2,s2 = strings[1] #get lowest
        
        del strings[0]
        del strings[0]
        
        strings.append((p1+p2, ((p1,s1),(p2,s2)) ))
    
    return strings[0]

tree = build_tree([(.95,['s1']), (.02,['s2']), (.03,['s3']) ])
  

Then, we have a top down traversal that assigns codes:

In [16]:
def assign_codes(tree, code=''):
    
    p,s = tree
    
    if len(s) == 1:
       print(s, code)
    else:
        assign_codes(s[0],code+'1')
        assign_codes(s[1],code+'0')

assign_codes(tree)

['s2'] 11
['s3'] 10
['s1'] 0
